In [13]:
import serial.tools.list_ports
ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p)

COM5 - USB-SERIAL CH340 (COM5)
COM4 - USB Serial Port (COM4)


In [15]:
import serial
import time
ser = serial.Serial('COM5', 115200, timeout=10)

In [11]:
ser.close()

In [41]:
#test echo (lab2)
ser.write(b'ABCD EFGHI J')
time.sleep(.001)
print(ser.read(12))
#works again after ser close + reset on pynq -> check yusuf testbench for mismatch rx and tx

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))

In [17]:
#test mp adder (lab3) (problem: received missing first bit + small bit flips)
import random
import time

OPERAND_WIDTH = 512

N_BYTES = int(OPERAND_WIDTH/8)

A = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)
B = random.randrange(2**(OPERAND_WIDTH-1), 2**OPERAND_WIDTH-1)

res = A + B

print("A     = ", hex(A))
print("B     = ", hex(B))
print("A + B = ", hex(res))

A_bytes = bytearray.fromhex(format(A, 'x'))
B_bytes = bytearray.fromhex(format(B, 'x'))

for op_byte in A_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

for op_byte in B_bytes:
    hex_byte = ("{0:02x}".format(op_byte))
    ser.write(bytearray.fromhex(hex_byte))
    time.sleep(0.001)

res_rcvd = ser.read(N_BYTES+1) 

res2 = int.from_bytes(res_rcvd, "big")

if res2==res:
    print("Result received correctly!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))
else:
    print("Result INCORRECT!")
    print("Expected   = ", hex(res))
    print("Received   = ", hex(res2))

A     =  0xae3a3839673451a54a0d237f166d01e001f2a7563b59dfc6ea5ff9804cccd5fce434f595bef5a34c8e447b899f71fb2bca8f51762fefc1920a16dab4b6cf5ca2
B     =  0xf352590ab7df9932d9956a60ce444ef128641978c8ae63e9a141e6cd18e0c1491edc988f075e3032aa1306b93e4237ed117bf07fd233bdd766711a311b84643e
A + B =  0x1a18c91441f13ead823a28ddfe4b150d12a56c0cf040843b08ba1e04d65ad974603118e24c653d37f38578242ddb43318dc0b41f602237f697087f4e5d253c0e0
Result INCORRECT!
Expected   =  0x1a18c91441f13ead823a28ddfe4b150d12a56c0cf040843b08ba1e04d65ad974603118e24c653d37f38578242ddb43318dc0b41f602237f697087f4e5d253c0e0
Received   =  0x1a18c91441f1383d823a28ddfe4b150d12a56c0cf040843b08ba1e04db179974603118e24c653d37ec65782431bb43318ed0b41f6022340407087f4e5d253c0e0


In [31]:
# Convert to hex strings without '0x' and pad with zeros if needed
hex_expected = hex(res)[2:].zfill(8)
hex_received = hex(res2)[2:].zfill(8)

print("Show Differences:")
print("Hex Expected = ", hex_expected)
print("Hex Received = ", hex_received)

    # Highlight mismatches with '^'
diff_string = ''.join(
        '^' if a != b else ' ' for a, b in zip(hex_expected, hex_received)
    )

print("Difference   = ", diff_string)

Show Differences:
Hex Expected =  1a18c91441f13ead823a28ddfe4b150d12a56c0cf040843b08ba1e04d65ad974603118e24c653d37f38578242ddb43318dc0b41f602237f697087f4e5d253c0e0
Hex Received =  1a18c91441f1383d823a28ddfe4b150d12a56c0cf040843b08ba1e04db179974603118e24c653d37ec65782431bb43318ed0b41f6022340407087f4e5d253c0e0
Difference   =               ^^                                          ^^^^                   ^^^     ^^^      ^^          ^^^^                
